# Загрузка данных в Qdrant

Этот ноутбук:
1. Удаляет все существующие документы из Qdrant
2. Загружает данные из JSON файла с сообщениями
3. Добавляет документы в векторную базу данных
4. Проверяет корректность загрузки


## 1. Импорты и настройка


In [13]:
import json
import logging
import sys
from pathlib import Path
from datetime import datetime

# Добавляем путь к проекту
project_root = Path("/srv/nlp1/eval_dir/T-bank_NLP_")
sys.path.insert(0, str(project_root))

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

print("=" * 80)
print(f"📦 ЗАГРУЗКА ДАННЫХ В QDRANT - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)


📦 ЗАГРУЗКА ДАННЫХ В QDRANT - 2025-11-17 00:46:02


## 2. Инициализация RetrieverService


In [14]:
from tplexity.retriever.retriever_service import RetrieverService

print("🔄 Инициализация RetrieverService...")
service = RetrieverService()
print("✅ RetrieverService инициализирован")


2025-11-17 00:46:02,841 - INFO - 🔄 [retriever_service] Инициализация гибридного поисковика
2025-11-17 00:46:02,842 - INFO - 🔄 [retriever][vector_search] Инициализация клиента Qdrant


🔄 Инициализация RetrieverService...


2025-11-17 00:46:06,976 - INFO - HTTP Request: GET https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333 "HTTP/1.1 200 OK"
2025-11-17 00:46:06,978 - INFO - ✅ [retriever][vector_search] Клиент Qdrant инициализирован: 8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333
2025-11-17 00:46:06,978 - INFO - ✅ [retriever][vector_search] Dense модель инициализирована, размерность: 1536
2025-11-17 00:46:06,979 - INFO - ✅ [retriever][vector_search] BM25 модель инициализирована


✅ RetrieverService инициализирован


## 3. Проверка количества документов (до очистки)


In [15]:
count_before = await service.vector_search.client.count(
    collection_name=service.vector_search.collection_name
)
print(f"📊 Документов в коллекции '{service.vector_search.collection_name}' (до очистки): {count_before.count}")


2025-11-17 00:46:07,123 - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/count "HTTP/1.1 200 OK"


📊 Документов в коллекции 'documents' (до очистки): 1470


## 4. Удаление всех документов


In [16]:
print("⚠️  Удаление всех документов из Qdrant...")
await service.delete_all_documents()
print("✅ Все документы удалены")

# Проверяем, что коллекция пуста
count_after_delete = await service.vector_search.client.count(
    collection_name=service.vector_search.collection_name
)
print(f"📊 Документов после очистки: {count_after_delete.count}")


2025-11-17 00:46:07,131 - WARNING - ⚠️ [retriever_service] Удаление всех документов
2025-11-17 00:46:07,132 - WARNING - ⚠️ [retriever][vector_search] Удаление всех документов из коллекции


⚠️  Удаление всех документов из Qdrant...


2025-11-17 00:46:07,201 - INFO - HTTP Request: DELETE https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents "HTTP/1.1 200 OK"
2025-11-17 00:46:07,203 - INFO - ✅ [retriever][vector_search] Коллекция documents удалена
2025-11-17 00:46:07,247 - INFO - HTTP Request: GET https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections "HTTP/1.1 200 OK"
2025-11-17 00:46:07,492 - INFO - HTTP Request: PUT https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents "HTTP/1.1 200 OK"
2025-11-17 00:46:07,493 - INFO - ✅ [retriever][vector_search] Коллекция documents создана с dense и sparse векторами
2025-11-17 00:46:07,494 - INFO - ✅ [retriever][vector_search] Коллекция documents пересоздана
2025-11-17 00:46:07,494 - INFO - ✅ [retriever_service] Все документы удалены
2025-11-17 00:46:07,539 - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-c

✅ Все документы удалены
📊 Документов после очистки: 0


## 5. Загрузка данных из JSON


In [17]:
# Путь к JSON файлу с сообщениями
messages_path = project_root / "src/eval/chunks_output.json"

print(f"🔄 Загрузка сообщений из {messages_path.name}...")
with open(messages_path, "r", encoding="utf-8") as f:
    messages = json.load(f)

print(f"✅ Загружено {len(messages)} сообщений из файла")
print(f"\n📋 Пример первого сообщения:")
print(f"  ID: {messages[0]['id']}")
print(f"  Channel ID: {messages[0]['channel_id']}")
print(f"  Date: {messages[0].get('date', 'N/A')}")
print(f"  Text: {messages[0].get('text', '')[:100]}...")


🔄 Загрузка сообщений из chunks_output.json...
✅ Загружено 1702 сообщений из файла

📋 Пример первого сообщения:
  ID: 7078
  Channel ID: 1418181070
  Date: 2025-11-14T12:14:02+00:00
  Text: **Вместе мы сильнее: запустили сообщество для инвесторов ****❤️**

Пока оно доступно в обновлённом п...


## 6. Подготовка данных для индексации


In [18]:
print("🔄 Подготовка документов для индексации...")

documents = []
metadatas = []

for msg in messages:
    text = msg.get("text", "").strip()
    if not text:
        continue
    
    documents.append(text)
    
    # Метаданные (составной ID для связи с ground truth)
    metadata = {
        "doc_id": f"{msg['channel_id']}_{msg['id']}",  # Составной ID
        "message_id": msg["id"],
        "channel_id": msg["channel_id"],
        "date": msg.get("date", ""),
        "link": msg.get("link", ""),
    }
    metadatas.append(metadata)

print(f"✅ Подготовлено {len(documents)} документов для индексации")
print(f"📊 Пропущено {len(messages) - len(documents)} пустых сообщений")


🔄 Подготовка документов для индексации...
✅ Подготовлено 1702 документов для индексации
📊 Пропущено 0 пустых сообщений


## 7. Добавление документов в Qdrant


In [19]:
print("🔄 Загрузка документов в Qdrant...")
print(f"⏳ Это может занять несколько минут...")

# Добавляем документы (UUID будут сгенерированы автоматически)
await service.add_documents(documents=documents, metadatas=metadatas)

print("✅ Документы успешно добавлены в Qdrant")


2025-11-17 00:46:07,598 - INFO - 🔄 [retriever_service] Добавление 1702 новых документов


🔄 Загрузка документов в Qdrant...
⏳ Это может занять несколько минут...


Batches: 100%|██████████| 54/54 [01:03<00:00,  1.17s/it]
2025-11-17 00:47:19,361 - INFO - HTTP Request: GET https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections "HTTP/1.1 200 OK"
2025-11-17 00:47:19,363 - INFO - ✅ [retriever][vector_search] Коллекция documents уже существует
2025-11-17 00:47:20,286 - INFO - HTTP Request: PUT https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points?wait=true "HTTP/1.1 200 OK"
2025-11-17 00:47:20,288 - INFO - 📦 [retriever][vector_search] Загружено 10/1702 документов
2025-11-17 00:47:20,857 - INFO - HTTP Request: PUT https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points?wait=true "HTTP/1.1 200 OK"
2025-11-17 00:47:20,859 - INFO - 📦 [retriever][vector_search] Загружено 20/1702 документов
2025-11-17 00:47:21,267 - INFO - HTTP Request: PUT https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.a

✅ Документы успешно добавлены в Qdrant


## 8. Проверка количества документов (после загрузки)


In [20]:
count_after_load = await service.vector_search.client.count(
    collection_name=service.vector_search.collection_name
)

print("=" * 80)
print("📊 ИТОГОВАЯ СТАТИСТИКА")
print("=" * 80)
print(f"📈 Документов до очистки:     {count_before.count}")
print(f"🗑️  Документов после очистки:  {count_after_delete.count}")
print(f"✅ Документов после загрузки:  {count_after_load.count}")
print(f"📦 Добавлено документов:       {len(documents)}")
print("=" * 80)

if count_after_load.count == len(documents):
    print("✅ Все документы успешно загружены!")
else:
    print(f"⚠️  Расхождение: ожидалось {len(documents)}, загружено {count_after_load.count}")


2025-11-17 00:47:50,685 - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/count "HTTP/1.1 200 OK"


📊 ИТОГОВАЯ СТАТИСТИКА
📈 Документов до очистки:     1470
🗑️  Документов после очистки:  0
✅ Документов после загрузки:  1702
📦 Добавлено документов:       1702
✅ Все документы успешно загружены!


## 9. Тестовый поиск


In [21]:
print("\n🔍 Тестовый поиск для проверки работоспособности...\n")

test_query = "инвестиции акции"
results = await service.search(test_query, top_k=5, top_n=3, use_rerank=False)

print(f"Запрос: '{test_query}'")
print(f"Найдено результатов: {len(results)}\n")

for i, (doc_id, score, text, metadata) in enumerate(results, 1):
    print(f"Результат {i}:")
    print(f"  Doc ID: {doc_id}")
    print(f"  Score: {score:.3f}")
    print(f"  Text: {text[:150]}...")
    print(f"  Metadata doc_id: {metadata.get('doc_id', 'N/A')}")
    print(f"  Link: {metadata.get('link', 'N/A')}")
    print()


2025-11-17 00:47:50,693 - INFO - 🔍 [retriever_service] Начало поиска для запроса: инвестиции акции...



🔍 Тестовый поиск для проверки работоспособности...



Batches: 100%|██████████| 1/1 [00:00<00:00, 14.82it/s]
2025-11-17 00:47:50,820 - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/query "HTTP/1.1 200 OK"
2025-11-17 00:47:50,823 - INFO - ✅ [retriever_service] Гибридный поиск завершен, найдено результатов: 5
2025-11-17 00:47:50,823 - INFO - ✅ [retriever_service] Поиск завершен, возвращено 3 результатов


Запрос: 'инвестиции акции'
Найдено результатов: 3

Результат 1:
  Doc ID: 45a56b64-610e-4214-821a-d9fae72ebd28
  Score: 0.500
  Text: **Знаете игру «Алхимию», где нужно смешивать простые элементы, создавая новое? Альфа-Инвестиции сыграли в неё с инвесторами — и выиграли все ****🎆**

...
  Metadata doc_id: 1418181070_6696
  Link: https://t.me/alfa_investments/6696

Результат 2:
  Doc ID: f69f510a-ed9a-47ba-9ce6-4f663fd8f8f0
  Score: 0.500
  Text: **Альфа-Инвестиции + Промомед = рекордная сделка с облигациями нового формата ****😍**

Альфа-Инвестиции снова подтвердили статус лидера альтернативных...
  Metadata doc_id: 1418181070_6587
  Link: https://t.me/alfa_investments/6587

Результат 3:
  Doc ID: 02534604-203c-4248-8ab7-df76bea8bbc5
  Score: 0.333
  Text: 🟥 Будут интересны и облигации. Например, ОФЗ могут сделать портфель стабильнее и вырасти в цене при смягчении денежно-кредитной политики за счёт роста...
  Metadata doc_id: 1418181070_6798
  Link: https://t.me/alfa_investments/6798



## 10. Проверка конкретного документа по ground truth ID


In [22]:
# Проверяем, что можем найти конкретный документ
print("\n🔍 Проверка поиска конкретного документа...\n")

# Берем первое сообщение из нашего датасета
first_msg = messages[0]
expected_doc_id = f"{first_msg['channel_id']}_{first_msg['id']}"

print(f"Ожидаемый doc_id: {expected_doc_id}")
print(f"Текст документа: {first_msg.get('text', '')[:150]}...\n")

# Ищем по тексту из этого документа
search_text = first_msg.get('text', '')[:100]  # Берем первые 100 символов
results = await service.search(search_text, top_k=5, top_n=5, use_rerank=False)

print(f"Результаты поиска (топ-5):")
found = False
for i, (doc_id, score, text, metadata) in enumerate(results, 1):
    metadata_doc_id = metadata.get('doc_id', '')
    is_target = metadata_doc_id == expected_doc_id
    marker = "✅ [ЦЕЛЕВОЙ ДОКУМЕНТ]" if is_target else ""
    
    print(f"  {i}. doc_id: {metadata_doc_id}, Score: {score:.3f} {marker}")
    
    if is_target:
        found = True

print()
if found:
    print("✅ Целевой документ найден! Система работает корректно.")
else:
    print("⚠️  Целевой документ не найден в топ-5. Это может быть нормально.")


2025-11-17 00:47:50,831 - INFO - 🔍 [retriever_service] Начало поиска для запроса: **Вместе мы сильнее: запустили сообщество для инве...



🔍 Проверка поиска конкретного документа...

Ожидаемый doc_id: 1418181070_7078
Текст документа: **Вместе мы сильнее: запустили сообщество для инвесторов ****❤️**

Пока оно доступно в обновлённом приложении Альфа-Инвестиций для андроида, но скоро ...



Batches: 100%|██████████| 1/1 [00:00<00:00, 49.26it/s]
2025-11-17 00:47:50,910 - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/query "HTTP/1.1 200 OK"
2025-11-17 00:47:50,912 - INFO - ✅ [retriever_service] Гибридный поиск завершен, найдено результатов: 5
2025-11-17 00:47:50,912 - INFO - ✅ [retriever_service] Поиск завершен, возвращено 5 результатов


Результаты поиска (топ-5):
  1. doc_id: 1418181070_7078, Score: 1.000 ✅ [ЦЕЛЕВОЙ ДОКУМЕНТ]
  2. doc_id: 1239405989_10908, Score: 0.333 
  3. doc_id: 1418181070_6958, Score: 0.333 
  4. doc_id: 1344086554_12388, Score: 0.250 
  5. doc_id: 1418181070_6942, Score: 0.250 

✅ Целевой документ найден! Система работает корректно.


## Готово! 🎉

Данные успешно загружены в Qdrant. Теперь можно запускать evaluation ноутбуки для оценки качества поиска.
